## Libraries

In [1]:
import numpy as np 
import pandas as pd
from pandas import DataFrame
from scipy.interpolate import interp1d
import pickle

## Import data

In [2]:
# Read in data
file = "Metro_zhvi.csv"
df = pd.read_csv(file)
df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31
0,102001,0,United States,country,NaN,128276.0,128617.0,128973.0,129698.0,130427.0,...,338512.0,344016.0,348988.0,353138.0,355519.0,356556.0,356904.0,357416.0,357905.0,357906.0
1,394913,1,"New York, NY",msa,NY,226266.0,227618.0,228829.0,231213.0,233418.0,...,594875.0,602165.0,609829.0,615934.0,620305.0,622586.0,623535.0,623959.0,624928.0,626122.0
2,753899,2,"Los Angeles, CA",msa,CA,240598.0,241457.0,242727.0,245142.0,247707.0,...,952694.0,973118.0,990735.0,992208.0,988839.0,973842.0,963691.0,953568.0,948645.0,945424.0
3,394463,3,"Chicago, IL",msa,IL,170254.0,170658.0,171176.0,172211.0,173301.0,...,303341.0,306934.0,311021.0,314088.0,315978.0,316308.0,316606.0,317244.0,317759.0,317300.0
4,394514,4,"Dallas, TX",msa,TX,129923.0,130024.0,130113.0,130326.0,130543.0,...,366987.0,376446.0,385132.0,392651.0,394824.0,393831.0,392135.0,392285.0,392143.0,390257.0


## Format df

In [3]:
# drop unneeded columns
df = df.drop(["RegionID","SizeRank","RegionType","StateName"], axis=1)

# remove spaces and commas from region name (bc they will become column names)
df["RegionName"] = df["RegionName"].str.replace(' ', '') 
df["RegionName"] = df["RegionName"].str.replace(',', '') 

# make RegionName the row index
df = df.set_index('RegionName')
# and drop the index column name
df.index.name = None

# transpose df
df = DataFrame.transpose(df)

# the date is the index now so turn it into a column
df["date"] = df.index
# and reset index
df = df.reset_index(drop=True)
# force date to be the first column
df.insert(0, 'date', df.pop("date"))

# make date into datetime type
df["date"] = pd.to_datetime(df["date"])

In [4]:
# Dimension of the data
print(df.shape)

# Data types
print(df.dtypes)

df.head()

(276, 894)
date            datetime64[ns]
UnitedStates           float64
NewYorkNY              float64
LosAngelesCA           float64
ChicagoIL              float64
                     ...      
ZapataTX               float64
KetchikanAK            float64
CraigCO                float64
LamesaTX               float64
VernonTX               float64
Length: 894, dtype: object


,date,UnitedStates,NewYorkNY,LosAngelesCA,ChicagoIL,DallasTX,HoustonTX,WashingtonDC,MiamiFL,PhiladelphiaPA,...,PecosTX,AtchisonKS,SpencerIA,SweetwaterTX,VermillionSD,ZapataTX,KetchikanAK,CraigCO,LamesaTX,VernonTX
0,2000-01-31,128276.0,226266.0,240598.0,170254.0,129923.0,127011.0,200209.0,140286.0,131271.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102246.0,NaN,NaN
1,2000-02-29,128617.0,227618.0,241457.0,170658.0,130024.0,127075.0,200484.0,140825.0,131708.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102636.0,NaN,NaN
2,2000-03-31,128973.0,228829.0,242727.0,171176.0,130113.0,126950.0,200883.0,141306.0,131976.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103269.0,NaN,NaN
3,2000-04-30,129698.0,231213.0,245142.0,172211.0,130326.0,127064.0,201744.0,142247.0,132634.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104029.0,NaN,NaN
4,2000-05-31,130427.0,233418.0,247707.0,173301.0,130543.0,127071.0,202922.0,143018.0,133157.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104795.0,NaN,NaN


## Deal with missing data

In [5]:
# length of time series is 276 months

# Missing values per column
print(df.isna().sum())

date              0
UnitedStates      0
NewYorkNY         0
LosAngelesCA      0
ChicagoIL         3
               ... 
ZapataTX        117
KetchikanAK      64
CraigCO           2
LamesaTX        136
VernonTX        114
Length: 894, dtype: int64


#### Fill missing

In [6]:
# Note: time series are allowed to be shorter than the full time frame

# Linear Interpolation    
df['rownum'] = np.arange(df.shape[0]) # add row number column
# repeat interpolation process for every region
for region in df.columns[1:-1]:
    df_nona = df[[region, 'rownum']].dropna()
    f = interp1d(df_nona['rownum'], df_nona[region], bounds_error=False) # allowe
    df[region] = f(df['rownum'])
df = df.drop('rownum', axis = 1) # drop the row number column we made

In [7]:
# Missing values per column
print(df.isna().sum())

# Remaining missing are from some time series being shorter

date              0
UnitedStates      0
NewYorkNY         0
LosAngelesCA      0
ChicagoIL         0
               ... 
ZapataTX        117
KetchikanAK      64
CraigCO           0
LamesaTX        135
VernonTX        113
Length: 894, dtype: int64


In [10]:
df.head()

,date,UnitedStates,NewYorkNY,LosAngelesCA,ChicagoIL,DallasTX,HoustonTX,WashingtonDC,MiamiFL,PhiladelphiaPA,...,PecosTX,AtchisonKS,SpencerIA,SweetwaterTX,VermillionSD,ZapataTX,KetchikanAK,CraigCO,LamesaTX,VernonTX
0,2000-01-31,128276.0,226266.0,240598.0,170254.0,129923.0,127011.0,200209.0,140286.0,131271.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102246.0,NaN,NaN
1,2000-02-29,128617.0,227618.0,241457.0,170658.0,130024.0,127075.0,200484.0,140825.0,131708.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102636.0,NaN,NaN
2,2000-03-31,128973.0,228829.0,242727.0,171176.0,130113.0,126950.0,200883.0,141306.0,131976.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103269.0,NaN,NaN
3,2000-04-30,129698.0,231213.0,245142.0,172211.0,130326.0,127064.0,201744.0,142247.0,132634.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104029.0,NaN,NaN
4,2000-05-31,130427.0,233418.0,247707.0,173301.0,130543.0,127071.0,202922.0,143018.0,133157.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104795.0,NaN,NaN


## Pickle the cleaned dataframe

In [11]:
df.to_pickle("cleaned_df.pkl")